In [1]:
import datetime 
from datetime import datetime
import tensorflow as tf

2021-12-10 21:37:51.902836: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary, Tokenizer, AdamWarmup, calc_train_steps

In [3]:
from sklearn.preprocessing import MultiLabelBinarizer

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
import pandas as pd
import numpy as np
import re

In [6]:
df_2006 = pd.read_json('/home/chikkamath/my_tensorflow/uspto2M_data/2006_USPTO.json')
df_2007 = pd.read_json('/home/chikkamath/my_tensorflow/uspto2M_data/2007_USPTO.json')
df_2008 = pd.read_json('/home/chikkamath/my_tensorflow/uspto2M_data/2008_USPTO.json')
df_2009 = pd.read_json('/home/chikkamath/my_tensorflow/uspto2M_data/2009_USPTO.json')
df_2010 = pd.read_json('/home/chikkamath/my_tensorflow/uspto2M_data/2010_USPTO.json')
df_2011 = pd.read_json('/home/chikkamath/my_tensorflow/uspto2M_data/2011_USPTO.json')
df_2012 = pd.read_json('/home/chikkamath/my_tensorflow/uspto2M_data/2012_USPTO.json')
df_2013 = pd.read_json('/home/chikkamath/my_tensorflow/uspto2M_data/2013_USPTO.json')
df_2014 = pd.read_json('/home/chikkamath/my_tensorflow/uspto2M_data/2014_USPTO.json')
df_2015 = pd.read_json('/home/chikkamath/my_tensorflow/uspto2M_data/2015_USPTO.json')



df = pd.concat([df_2006, df_2007, df_2008, df_2009, df_2010, df_2011, df_2012, df_2013, df_2014, df_2015], axis=0, ignore_index=True)

# df = pd.read_json('/home/chikkamath/my_tensorflow/uspto2M_data/2006_USPTO.json')

In [7]:
print(df.shape)
df.head()

(2000147, 4)


,Subclass_labels,Abstract,Title,No
0,[G11B],a magnetic disc unit having a rotating disc ty...,magnetic disc unit with range of gap between d...,US07072141
1,[A41D],methods and systems for transforming a volume ...,systems and methods for transformable suits,US06981282
2,[A41D],a gardening glove or the like designed to prev...,gardening gloves with cushion insert to protec...,US06981283
3,[A63B],a hockey helmet for receiving a head of a wear...,hockey helmet comprising a lateral adjustment ...,US06981284
4,[E03D],a toilet and toilet system suitable for a boat...,toilet and method of operation,US06981285


In [8]:
df = df[reversed(df.columns)]
# df = df.explode('Subclass_labels')
#df = df[:500000] # experiment 
df.shape

(2000147, 4)

In [9]:
abstracts = df.Abstract.values
titles = df.Title.values

In [10]:
for i in range(len(abstracts)):
  abstracts[i] = titles[i] + ' : ' + abstracts[i]

In [11]:
df.Abstract = abstracts

In [12]:
df.head()

,No,Title,Abstract,Subclass_labels
0,US07072141,magnetic disc unit with range of gap between d...,magnetic disc unit with range of gap between d...,[G11B]
1,US06981282,systems and methods for transformable suits,systems and methods for transformable suits : ...,[A41D]
2,US06981283,gardening gloves with cushion insert to protec...,gardening gloves with cushion insert to protec...,[A41D]
3,US06981284,hockey helmet comprising a lateral adjustment ...,hockey helmet comprising a lateral adjustment ...,[A63B]
4,US06981285,toilet and method of operation,toilet and method of operation : a toilet and ...,[E03D]


In [13]:
#y_binarized[4]

In [14]:
type(df["Subclass_labels"][0])

list

In [15]:
# converting into tuple
#df.Subclass_labels = df.Subclass_labels.apply(lambda x: tuple(re.sub("\[|\]|\'", '', x).split(', ')))


In [16]:
VOCAB_PATH = '/home/chikkamath/my_tensorflow/bert_for_patents_vocab_39k.txt'
vocab_ = load_vocabulary(vocab_path=VOCAB_PATH)

In [17]:
tokenizer = Tokenizer(vocab_)

In [18]:
# define sequence length for tokenizer 
SEQ_LEN = 64  # finished 128

In [19]:
#def setup_training_data(training_set, labels):
#  indices = []
#  targets = []
#  MAX_SEQ_LEN = SEQ_LEN
#  for i in range(len(training_set)):
#    ids, segments = tokenizer.encode(training_set[i], max_len=MAX_SEQ_LEN)
#    indices.append(ids)
#    targets.append(labels[i])
#    items = list(zip(indices, targets))
#  np.random.shuffle(items)
#  indices, targets = zip(*items)
#  indices = np.array(indices)
#  return [indices, np.zeros_like(indices)], np.array(targets)

In [20]:
def setup_training_data(training_set, labels):
  indices = []
  targets = []
  MAX_SEQ_LEN = SEQ_LEN

  for i in range(len(training_set)):
    ids, segments = tokenizer.encode(training_set[i], max_len=MAX_SEQ_LEN)
    indices.append(ids)
    targets.append(labels[i])
  return indices, np.array(targets)  

In [21]:
X = df.Abstract.values
y = df['Subclass_labels'].values

In [22]:
#print(y)

In [23]:
mlb = MultiLabelBinarizer()
y_binarized = mlb.fit_transform(y)

In [24]:
print(y_binarized.shape)

(2000147, 632)


In [25]:
print(type(y_binarized))

<class 'numpy.ndarray'>


In [26]:
# creating data and labels for futhert training 
start = datetime.now()
data, labels = setup_training_data(X, y_binarized)
end = datetime.now()
print(end-start)

0:34:03.302412


In [27]:
# checking shape in future we can remove it 
print(len(data))
print(labels.shape)

2000147
(2000147, 632)


In [28]:
print(data[0])

[2, 7725, 5525, 2796, 1672, 2511, 1662, 6243, 1755, 5525, 1663, 30574, 1029, 1042, 7725, 5525, 2796, 2048, 1042, 13283, 5525, 2493, 7725, 5525, 1042, 1797, 1670, 3070, 1663, 16025, 13792, 13859, 1740, 2616, 1665, 1663, 1678, 1661, 7725, 5525, 1663, 1042, 30574, 3858, 1684, 1868, 2967, 1662, 1684, 5723, 22940, 1662, 1661, 7725, 5525, 2937, 1042, 1777, 1738, 1042, 8783, 2514, 1668, 3]


In [29]:
tv_size = df.shape[0] - df_2015.shape[0]
print(tv_size)

1950247


In [30]:
# Test set 
x_test = data[tv_size:]
y_test = labels[tv_size:]

# Checking shapes
print(len(x_test))
print(y_test.shape)

49900
(49900, 632)


In [61]:
# data fro train and validaiton 

X = data[:tv_size]
Y = labels[:tv_size]

In [96]:
#Experimnets
X = data[:400]
Y = labels[:400]

In [97]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.05, random_state=101)

In [98]:
#X_test, X_val, y_test, y_val = train_test_split(X_testt, y_testt, test_size=0.5, random_state=101)

In [99]:
# checking shape in future we can remove it 
print(len(X_train))
print(len(x_test))
print(y_train.shape)
print(y_test.shape)
print(len(X_val))
print(y_val.shape)

380
49900
(380, 632)
(49900, 632)
20
(20, 632)


In [100]:
# since input takes two values we are creating such input 
#X_train_in = [np.array(X_train), np.zeros_like(X_train)]

In [101]:
# since input takes two values we are creating such input 
#X_test_in = [np.array(X_test), np.zeros_like(X_test)]

In [102]:
# checking shape in future we can remove it
print(X_train_in[0].shape)
print(X_train_in[1].shape)

print(X_test_in[0].shape)
print(X_test_in[1].shape)

NameError: name 'X_train_in' is not defined

In [103]:
BATCH_SIZE = 32 # 32

In [104]:
X_train_np = np.array(X_train)

In [105]:
#X_val_np = np.array(X_val)

In [106]:
print(X_train_np.shape)
#print(X_val_np.shape)

(380, 64)


In [107]:
def generator_n(data, classes, batch_size=BATCH_SIZE):
  #num_samples = data.shape[0]
  num_samples = len(data)

  while True:
    
    idx = np.random.permutation(len(data))
    print(idx)
    data, classes = data[idx], classes[idx]
    
    for i in range(0, num_samples, batch_size):

      batch_samples_data = data[i:i+batch_size]
      batch_samples_data = [np.array(batch_samples_data), np.zeros_like(batch_samples_data)]
      #batch_samples_data = tf.convert_to_tensor(batch_samples_data) 
        
      batch_samples_class = classes[i:i+batch_size]  
      #batch_samples_class = batch_samples_class.fillna(0)
      #batch_samples_class = batch_samples_class.to_numpy()
      #batch_samples_class = tf.convert_to_tensor(batch_samples_class)
      #print(batch_samples_data)
      #print(batch_samples_class)
      yield batch_samples_data, batch_samples_class

In [125]:
def generator_n_val(data, classes, batch_size=BATCH_SIZE):
  #num_samples = data.shape[0]
  num_samples = len(data)

  while True:
    
    for i in range(0, num_samples, batch_size):

      batch_samples_data = data[i:i+batch_size]
      batch_samples_data = [np.array(batch_samples_data), np.zeros_like(batch_samples_data)]
      #batch_samples_data = tf.convert_to_tensor(batch_samples_data) 
        
      batch_samples_class = classes[i:i+batch_size]  
      #batch_samples_class = batch_samples_class.fillna(0)
      #batch_samples_class = batch_samples_class.to_numpy()
      #batch_samples_class = tf.convert_to_tensor(batch_samples_class)
      #print(batch_samples_data)
      #print(batch_samples_class)
      yield batch_samples_data, batch_samples_class

In [126]:
# train generator
train_gen = generator_n(X_train_np, y_train)

# validation generator
validation_gen = generator_n_val(X_val, y_val)

In [127]:
#next(train_gen)

In [128]:
#next(validation_gen)

## Loading model to run 

In [129]:
CONFIG_PATH = '/home/chikkamath/my_tensorflow/bert_for_patents_large_config.json'
CHECKPOINT_PATH = '/home/chikkamath/my_tensorflow/checkpoint/temp_dir/raw/model.ckpt'

In [130]:
BATCH_SIZE = 32 # PRVIOUS 4
EPOCHS = 1
LR = 1e-5

In [131]:
def load_pretrained_model(config_path=CONFIG_PATH,checkpoint=CHECKPOINT_PATH,max_seq_len=SEQ_LEN):
  print('Loading Pretrained model in progress....')
  model = load_trained_model_from_checkpoint(
      config_path,
      checkpoint,
      training = True, # Change False to train just ...
      trainable= True, # ... Top layer of the model
      seq_len=max_seq_len,
  )
  print('Model loaded successfully.')
  return model

In [132]:
def prepare_keras_model(pretrained_model): # labels, , batch_size=BATCH_SIZE, epochs=EPOCHS, learning_rate = LR
  import keras
  inputs = pretrained_model.inputs[:2]
  dense = pretrained_model.get_layer('NSP-Dense').output
  #print(dense)
  #X = keras.layers.Dense(1024, activation='relu')  
  # X = keras.layers.Dropout(0.1)(dense)  
  outputs = keras.layers.Dense(y_binarized.shape[1], activation='sigmoid', kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(dense)

  # outputs = keras.layers.Dense(y_train_binarized.shape[1], activation='sigmoid')


  # decay_steps, warmup_steps = calc_train_steps(
  #     labels.shape[0],
  #     batch_size = 16,
  #     epochs=2
  # )

  model = keras.models.Model(inputs, outputs)
  # loss = 'mean_squared_error',  
  #  optimizer = 'sgd', metrics = [metrics.categorical_accuracy]
  
 
    
  model.compile(
      optimizer = 'Adam',
      loss='binary_crossentropy',
      metrics=['accuracy']
  )
  #model.summary()
  return model

In [133]:
#def start_training(model, epochs=EPOCHS, batch_size=BATCH_SIZE):
 # history = model.fit(
  #        X_train_in,
   #       y_train,
    #      epochs=epochs,
     #     batch_size=batch_size,
      #    validation_split=0.1
          # validation_data = (data_val,labels_val)
    #)
 # return history

In [134]:
#checkpoint_filepath = '/home/chikkamath/my_tensorflow/tmp/checkpoint'
#model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
#    filepath=checkpoint_filepath,
 #   save_weights_only=True,
  #  monitor='val_accuracy',
   # mode='max',
    #save_best_only=True) 

In [135]:
def start_training(model, epochs=EPOCHS, batch_size=BATCH_SIZE):
  history = model.fit(
          train_gen,
          steps_per_epoch=y_train.shape[0]//32,
          epochs=epochs,
          validation_data=validation_gen,
          validation_steps=y_test.shape[0]//32
          #callbacks=[model_checkpoint_callback] 
          # validation_data = (data_val,labels_val)
      )
  return history

In [136]:
pretrained_model = load_pretrained_model()

Loading Pretrained model in progress....
Model loaded successfully.


In [137]:
# Run this cell once you have trained the model with training = False 
# pretrained_model.trainable = True

In [138]:
pretrained_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 64)]         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 64)]         0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 64, 1024), ( 40815616    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 64, 1024)     2048        Input-Segment[0][0]              
____________________________________________________________________________________________

In [139]:
model = prepare_keras_model(pretrained_model=pretrained_model)

In [140]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 64)]         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 64)]         0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 64, 1024), ( 40815616    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 64, 1024)     2048        Input-Segment[0][0]              
____________________________________________________________________________________________

In [141]:
start = datetime.now()
history = start_training(model, epochs=EPOCHS)
end = datetime.now()

print(end-start)

[331 353 325 263  83  14 124  22 294 266 233 232 371  43  71 366 126 231
  28 239  60 370 364 166 260 277 165  31  99 243  77 297  96 216 348 285
 293 363  61 330 240 127  70 158  82 367 244 210  13 181 185  74  34 138
 116  75 180  87 154 169 224 102 132 287 347 249 202 187  95  26 334 176
  48 372 339 211 221 167 246 267 178 332 354  62 207  36  69 326 315 198
 236 195 261 338 253 106 309 271 279 129  37 168 173 156 365  50 148 189
 196 311 342 245 322 273 337 191 292  45 223 296 262 303 237 103 145 161
 206 250 378 255 118 313  35  73  49 259 248  56 369 360 217  39 226  30
  40 119 336 252 295 229  91 108 164 177 109 310   7  33 212 312 327  86
 278 101  66 115 142  15 171  10 135 254  52  81 286 357 215   2  58 379
  19 321 170 264 190 143   1  44 288 140  25 136 300 105 160 213  53 141
  90 268 362  20  21 131 204 359  93 117 114 110 355 209 225 186  18 149
 373  11 340 111 120 152  27 230 112 218 197 283  79  63 306  78  42 183
 208 182 307 133 220  29 222 139 163  17 361  68 25

2021-12-11 11:42:48.577094: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.0KiB (rounded to 4096)requested by op Fill
Current allocation summary follows.
2021-12-11 11:42:48.577433: I tensorflow/core/common_runtime/bfc_allocator.cc:972] BFCAllocator dump for GPU_0_bfc
2021-12-11 11:42:48.577463: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (256): 	Total Chunks: 202, Chunks in use: 202. 50.5KiB allocated for chunks. 50.5KiB in use in bin. 896B client-requested in use in bin.
2021-12-11 11:42:48.577478: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2021-12-11 11:42:48.577494: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (1024): 	Total Chunks: 1, Chunks in use: 1. 1.2KiB allocated for chunks. 1.2KiB in use in bin. 1.0KiB client-requested in use in bin.
2021-12-1

ResourceExhaustedError: in user code:

    /home/chikkamath/anaconda3/envs/bert_env/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/chikkamath/anaconda3/envs/bert_env/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/chikkamath/anaconda3/envs/bert_env/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/chikkamath/anaconda3/envs/bert_env/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/chikkamath/anaconda3/envs/bert_env/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
 

In [120]:
#start = datetime.now()
#history = start_training(model, epochs=EPOCHS)
#end = datetime.now()

#print(end-start)

Epoch 1/2
14062/14062 [==============================] - 23245s 2s/step - loss: 0.0089 - accuracy: 0.3832 - val_loss: 0.0061 - val_accuracy: 0.4952
Epoch 2/2
14062/14062 [==============================] - 23211s 2s/step - loss: 0.0063 - accuracy: 0.4851 - val_loss: 0.0060 - val_accuracy: 0.5055
12:54:16.171980


In [89]:
X_test_mod = [np.array(x_test), np.zeros_like(x_test)]

In [90]:
print(type(X_test_mod))
print(y_test.shape)

<class 'list'>
(49900, 632)


In [91]:
print(y_test[:5])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [92]:
y_pred = model.predict(X_test_mod)

In [108]:
K = tf.keras.backend # keras as backend
k = 1 # no. of p,r,f1

In [109]:
topk_values, topk_indices = tf.nn.top_k(y_pred, k, sorted=False, name="topk")

In [110]:
print(topk_values)
print(topk_indices)

tf.Tensor(
[[0.17273133]
 [0.17273133]
 [0.17273133]
 ...
 [0.17273133]
 [0.17273133]
 [0.17273133]], shape=(49900, 1), dtype=float32)
tf.Tensor(
[[504]
 [504]
 [504]
 ...
 [504]
 [504]
 [504]], shape=(49900, 1), dtype=int32)


In [111]:
ii, _ = tf.meshgrid(tf.range(y_test.shape[0]), tf.range(k), indexing='ij')

In [112]:
iip = ii.numpy()
print(iip)

[[    0]
 [    1]
 [    2]
 ...
 [49897]
 [49898]
 [49899]]


In [113]:
index_tensor = tf.reshape(tf.stack([ii, topk_indices], axis=-1), shape=(-1, 2))

In [114]:
prediction_tensor = tf.compat.v1.sparse_to_dense(sparse_indices=index_tensor, 
                                       output_shape=tf.shape(y_pred),
                                       default_value=0,
                                       sparse_values=1.0,
                                       validate_indices=False
                                       )

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [115]:
prediction_tensor = tf.cast(prediction_tensor, tf.keras.backend.floatx())

In [116]:
print(prediction_tensor.shape)

(49900, 632)


In [117]:
true_positive = K.sum(tf.multiply(prediction_tensor, y_test))

In [118]:
c2 = K.sum(prediction_tensor)
false_positive = c2 - true_positive

In [119]:
# CALCULATE PRECISION
pri=true_positive/(c2+K.epsilon())

In [120]:
pri_val = pri.numpy()
print(pri_val)

0.006733467


In [121]:
print(y_test.dtype)

int64


In [122]:
y_test_n = y_test.astype('float32')

In [123]:
# CALCULATE RECALL
c3 = K.sum(y_test_n)
rec = true_positive/(c3+K.epsilon())

In [124]:
rec_val = rec.numpy()
print(rec_val)

0.0034959943


In [81]:
# CALCULATE F1 SCORE
f1 = (2*pri*rec)/(pri+rec+K.epsilon())

In [82]:
f1_val = f1.numpy()
print(f1_val)

0.47726864


In [93]:
K = tf.keras.backend # keras as backend
k = 10 # no. of p,r,f1

In [94]:
topk_values, topk_indices = tf.nn.top_k(y_pred, k, sorted=False, name="topk")

In [95]:
print(topk_values)
print(topk_indices)

tf.Tensor(
[[0.17273133 0.10094172 0.08975704 ... 0.04273421 0.04104036 0.0390075 ]
 [0.17273133 0.10094172 0.08975704 ... 0.04273421 0.04104036 0.0390075 ]
 [0.17273133 0.10094172 0.08975704 ... 0.04273421 0.04104036 0.0390075 ]
 ...
 [0.17273133 0.10094172 0.08975704 ... 0.04273421 0.04104036 0.0390075 ]
 [0.17273133 0.10094172 0.08975704 ... 0.04273421 0.04104036 0.0390075 ]
 [0.17273133 0.10094172 0.08975704 ... 0.04273421 0.04104036 0.0390075 ]], shape=(49900, 10), dtype=float32)
tf.Tensor(
[[504 114 542 ... 625 503 163]
 [504 114 542 ... 625 503 163]
 [504 114 542 ... 625 503 163]
 ...
 [504 114 542 ... 625 503 163]
 [504 114 542 ... 625 503 163]
 [504 114 542 ... 625 503 163]], shape=(49900, 10), dtype=int32)


In [158]:
ii, _ = tf.meshgrid(tf.range(y_test.shape[0]), tf.range(k), indexing='ij')

In [159]:
iip = ii.numpy()
print(iip)

[[    0     0     0 ...     0     0     0]
 [    1     1     1 ...     1     1     1]
 [    2     2     2 ...     2     2     2]
 ...
 [49897 49897 49897 ... 49897 49897 49897]
 [49898 49898 49898 ... 49898 49898 49898]
 [49899 49899 49899 ... 49899 49899 49899]]


In [160]:
index_tensor = tf.reshape(tf.stack([ii, topk_indices], axis=-1), shape=(-1, 2))

In [161]:
prediction_tensor = tf.compat.v1.sparse_to_dense(sparse_indices=index_tensor, 
                                       output_shape=tf.shape(y_pred),
                                       default_value=0,
                                       sparse_values=1.0,
                                       validate_indices=False
                                       )

In [162]:
prediction_tensor = tf.cast(prediction_tensor, tf.keras.backend.floatx())

In [163]:
print(prediction_tensor.shape)

(49900, 632)


In [164]:
true_positive = K.sum(tf.multiply(prediction_tensor, y_test))

In [165]:
c2 = K.sum(prediction_tensor)
false_positive = c2 - true_positive

In [166]:
# CALCULATE PRECISION
pri=true_positive/(c2+K.epsilon())

In [167]:
pri_val = pri.numpy()
print(pri_val)

0.15440682


In [168]:
y_test_n = y_test.astype('float32')

In [169]:
# CALCULATE RECALL
c3 = K.sum(y_test_n)
rec = true_positive/(c3+K.epsilon())

In [170]:
rec_val = rec.numpy()
print(rec_val)

0.80167514


In [171]:
# CALCULATE F1 SCORE
f1 = (2*pri*rec)/(pri+rec+K.epsilon())

In [172]:
f1_val = f1.numpy()
print(f1_val)

0.2589403


## Calculate precision considring threshold 

In [193]:
y_pred_norm = [[1 if val >= 0.7 else 0 for val in pred]for pred in y_pred]
y_pred_norm = np.array(y_pred_norm)

In [194]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, hamming_loss, multilabel_confusion_matrix, jaccard_score, classification_report

In [195]:
y_pred_norm.shape

(49900, 632)

In [196]:
y_test.shape

(49900, 632)

In [197]:
print("F1-Score : {:.4f}".format(f1_score(y_test, y_pred_norm, average='samples')))
print("F1-Score : {:.4f}".format(f1_score(y_test, y_pred_norm, average='micro')))
print("hamming_loss : {:.4f}".format(hamming_loss(y_test, y_pred_norm)))
print("Precision-Score : {:.4f}".format(precision_score(y_test, y_pred_norm, average='micro')))
print("Recall-Score : {:.4f}".format(recall_score(y_test, y_pred_norm, average='micro')))

F1-Score : 0.3726
F1-Score : 0.3933
hamming_loss : 0.0024
Precision-Score : 0.8191
Recall-Score : 0.2588


In [214]:
print("Jaccard-score : {:.4f}".format(jaccard_score(y_test, y_pred_norm, average='micro')))

Jaccard-score : 0.1741


In [194]:
print((classification_report(y_test, y_pred_norm)))

              precision    recall  f1-score   support

           0       0.09      0.86      0.16        29
           1       0.05      0.83      0.10        23
           2       0.22      0.90      0.36        40
           3       0.00      0.00      0.00         7
           4       0.23      0.71      0.35        31
           5       0.97      0.94      0.95       488
           6       0.79      0.73      0.76        15
           7       0.67      0.39      0.50       114
           8       0.00      0.00      0.00         1
           9       0.52      0.73      0.61        15
          10       0.23      0.55      0.32       391
          11       0.00      0.00      0.00         9
          12       0.16      0.93      0.27        15
          13       0.00      0.00      0.00         4
          14       0.00      0.00      0.00         4
          15       0.33      0.17      0.22         6
          16       0.80      0.27      0.40        30
          17       0.36    

/home/chikkamath/anaconda3/envs/bert_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/chikkamath/anaconda3/envs/bert_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/chikkamath/anaconda3/envs/bert_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start